<a href="https://colab.research.google.com/github/RogerHeederer/FundamentalConceptOfDL/blob/master/2_AutoFeatureExtractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ref: 봉수골 개발자 이선비님의 깃허브를 참조하였으며, 학습용으로 진행한 테스트입니다

추가적으로 필요한 설명 및 요약들을 제 나름대로 표현하여 작성하였습니다.

##Flatten을 활용한 CNN 학습법

In [21]:
import tensorflow as tf
import pandas as pd

In [22]:
(독립변수, 종속변수), _ = tf.keras.datasets.mnist.load_data()
print(독립변수.shape, 종속변수.shape)

(60000, 28, 28) (60000,)


In [45]:
print(독립변수[0]) # 숫자 5을 28*28 그림으로 표현한 데이터가 독립

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   3  18  18  18 126 136 175  26 166 255
 247 127   0   0   0   0   0   0   0   0   0   0   0   0  30  36  94 154
 170 253 253 253 253 253 225 172 253 242 195  64   0   0   0   0   0   0
   0   0   0   0   0  49 238 253 253 253 253 253 253 253 253 251  93  82
  82  56  39   0   0   0   0   0   0   0   0   0   0   0   0  18 219 253
 253 253 253 253 198 182 247 241   0   0   0   0   

In [27]:
종속변수[0:20] # 종속 변수는 0~9까지의 range를 가지고 있고, 이 데이터는 6만개 있다.

array([5, 0, 4, 1, 9, 2, 1, 3, 1, 4, 3, 5, 3, 6, 1, 7, 2, 8, 6, 9],
      dtype=uint8)

데이터의 shape 조절은 reshape과 flatten 2가지 방식으로 할 수 있다

1. reshape으로 하는 방법

In [28]:
독립변수 = 독립변수.reshape(60000, 784)
종속변수 = pd.get_dummies(종속변수) # 0~9 범주에 있는 종속변수 데이터를 원핫 인코딩 하는 방식

In [31]:
독립변수.shape, 종속변수.shape

((60000, 784), (60000, 10))

In [46]:
종속변수 # 종속변수[0]을 보면, '5' 컬럼에 1로 처리되어 있고 나머지 값들엔 0으로 처리(원핫인코딩)

,0,1,2,3,4,5,6,7,8,9
0,0,0,0,0,0,1,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
59995,0,0,0,0,0,0,0,0,1,0
59996,0,0,0,1,0,0,0,0,0,0
59997,0,0,0,0,0,1,0,0,0,0
59998,0,0,0,0,0,0,1,0,0,0


## 모델 만들기

In [47]:
X = tf.keras.layers.Input(shape=[784]) # 28x28 -> 784로 위 데이터 구축 단계에서 reshape됨
H = tf.keras.layers.Dense(84, activation='swish')(X)
Y = tf.keras.layers.Dense(10, activation='softmax')(H)
model = tf.keras.models.Model(X, Y)
model.compile(loss='categorical_crossentropy', metrics = 'accuracy')

In [50]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 784)]             0         
_________________________________________________________________
dense (Dense)                (None, 84)                65940     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                850       
Total params: 66,790
Trainable params: 66,790
Non-trainable params: 0
_________________________________________________________________


784 노드 * 히든레이어 (노드84개) = 65,856개의 노드. 여기에 bias 84개 더하면 = 65940

즉 65,856 weight값과 84개의 bias들 ( = 파라미터들)을 학습한다.

##정리하자면, 손글씨를 가장 잘 분류할 수 있는 특징들(기준들) 84개를 추려내고자 한다.

## 그러기 위해 weight 65,856개와 84개의 bias들을 학습을 통해 최적의 값들을 찾고자 한다 ##

## 고로 해당 히든 레이어에서 시켜야 할 파라미터는 65,940개이다##

## 마지막 단계에서, 손글씨를 가장 잘 분류할 수 있는 특징 84개를 10개의 결과값(Y)에 이어준다.

## 여기서 발생되는 파라미터는 84 * 10 + bias 10개. 즉 850 개의 파라미터들도 학습시켜줘야 한다.

## 예측한 결과값들에 잘 이어줘야 하니깐~~

In [49]:
model.fit(독립변수, 종속변수, epochs=10)

Epoch 1/10
1875/1875 [==============================] - 5s 2ms/step - loss: 6.3616 - accuracy: 0.7964
Epoch 2/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.5581 - accuracy: 0.9102
Epoch 3/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.4313 - accuracy: 0.9288
Epoch 4/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.4168 - accuracy: 0.9326
Epoch 5/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.4129 - accuracy: 0.9398
Epoch 6/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3538 - accuracy: 0.9459
Epoch 7/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3696 - accuracy: 0.9442
Epoch 8/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3549 - accuracy: 0.9466
Epoch 9/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3379 - accuracy: 0.9494
Epoch 10/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.341

With Flatten

In [51]:
(독립변수, 종속변수), _ = tf.keras.datasets.mnist.load_data()
print(독립변수.shape, 종속변수.shape)

(60000, 28, 28) (60000,)


In [52]:
종속변수 = pd.get_dummies(종속변수)

In [53]:
print(독립변수.shape, 종속변수.shape)

(60000, 28, 28) (60000, 10)


데이터 단에서 reshape으로 조정해주지 않았기 때문에, 모델 구성에서 flatten을 활용해서 1차원으로 만들어준다>

In [55]:
X = tf.keras.layers.Input(shape=[28,28]) # 28x28 그대로 받아주고
H = tf.keras.layers.Flatten()(X) # 28x28을 784로 1차원으로 펴준다. [[..],[..]...[..]] --> [....]
H = tf.keras.layers.Dense(84, activation='swish')(H)
Y = tf.keras.layers.Dense(10, activation='softmax')(H)
model = tf.keras.models.Model(X, Y)
model.compile(loss='categorical_crossentropy', metrics='accuracy')

In [56]:
model.fit(독립변수, 종속변수, epochs=10)

Epoch 1/10
1875/1875 [==============================] - 4s 2ms/step - loss: 6.1619 - accuracy: 0.8010
Epoch 2/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.5541 - accuracy: 0.9133
Epoch 3/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.4622 - accuracy: 0.9261
Epoch 4/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.4154 - accuracy: 0.9368
Epoch 5/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3793 - accuracy: 0.9440
Epoch 6/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3526 - accuracy: 0.9439
Epoch 7/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3531 - accuracy: 0.9470
Epoch 8/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3480 - accuracy: 0.9469
Epoch 9/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3467 - accuracy: 0.9495
Epoch 10/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.332

In [57]:
# 모델 이용해보기
pred = model.predict(독립변수[0:5])

In [59]:
# 독립변수[0:5] -> (5, 28, 28): 28x28 차원을 가진 데이터 5개를 모델에 넣었더니, 
pred.shape #(5, 10) : 10차원을 가진 데이터 5개를 내뱉어준다.

(5, 10)

In [60]:
pd.DataFrame(pred).round(2) # 아래 답안과 일치하는 것을 확인할 수 있다.

,0,1,2,3,4,5,6,7,8,9
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [61]:
종속변수[0:5]

,0,1,2,3,4,5,6,7,8,9
0,0,0,0,0,0,1,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,1
